In [1]:
from dotenv import load_dotenv
import jikanpy
import os
import requests
import pandas as pd
load_dotenv()

secret = os.getenv("secret")
id = os.getenv("id")

jikan = jikanpy.Jikan()

In [2]:
username = 'jackljk'
req = requests.get(f"https://api.myanimelist.net/v2/users/{username}/animelist?limit=1000", headers={"X-MAL-CLIENT-ID": id, "client_secret": secret})

In [3]:
def get_anime_info(anime_id):
    jikan = jikanpy.Jikan()
    anime = jikan.anime(anime_id)['data']
    return anime

In [28]:
df_test = pd.DataFrame()

def test(anime_id):
    test = f"https://api.myanimelist.net/v2/anime/{anime_id}?fields=id,title,main_picture,alternative_titles,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,my_list_status,num_episodes,start_season,broadcast,source,average_episode_duration,rating,pictures,background,related_anime,related_manga,recommendations,studios,statistics'"
    reqtest = requests.get(test, headers={"X-MAL-CLIENT-ID": id, "client_secret": secret})
    
    return reqtest.json()

In [15]:
req = requests.get(f"https://api.myanimelist.net/v2/anime/ranking?ranking_type=airing&limit=10", headers={"X-MAL-CLIENT-ID": id, "client_secret": secret})

In [30]:
test(58567)

{'id': 58567,
 'title': 'Ore dake Level Up na Ken Season 2: Arise from the Shadow',
 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/1448/147351.jpg',
  'large': 'https://cdn.myanimelist.net/images/anime/1448/147351l.jpg'},
 'alternative_titles': {'synonyms': ['Solo Leveling Second Season'],
  'en': 'Solo Leveling Season 2: Arise from the Shadow',
  'ja': '俺だけレベルアップな件 Season 2 -Arise from the Shadow-'},
 'start_date': '2025-01-05',
 'synopsis': 'Second season of Ore dake Level Up na Ken.',
 'mean': 8.84,
 'rank': 28,
 'popularity': 918,
 'num_list_users': 278868,
 'num_scoring_users': 39713,
 'nsfw': 'white',
 'created_at': '2024-03-30T16:32:46+00:00',
 'updated_at': '2025-01-05T06:31:58+00:00',
 'media_type': 'tv',
 'status': 'currently_airing',
 'genres': [{'id': 1, 'name': 'Action'},
  {'id': 50, 'name': 'Adult Cast'},
  {'id': 2, 'name': 'Adventure'},
  {'id': 10, 'name': 'Fantasy'},
  {'id': 82, 'name': 'Urban Fantasy'}],
 'num_episodes': 13,
 'start_season': 

In [26]:
test = f"https://api.myanimelist.net/v2/anime/{58567}?fields=id,title,popularity,statistics,rank,mean,num_list_users,num_scoring_users,rating"
reqtest = requests.get(test, headers={"X-MAL-CLIENT-ID": id, "client_secret": secret})
reqtest.json()

{'id': 58567,
 'title': 'Ore dake Level Up na Ken Season 2: Arise from the Shadow',
 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/1448/147351.jpg',
  'large': 'https://cdn.myanimelist.net/images/anime/1448/147351l.jpg'},
 'popularity': 918,
 'statistics': {'status': {'watching': '155660',
   'completed': '101',
   'on_hold': '1422',
   'dropped': '288',
   'plan_to_watch': '121397'},
  'num_list_users': 278868},
 'rank': 28,
 'mean': 8.84,
 'num_list_users': 278868,
 'num_scoring_users': 39713,
 'rating': 'r'}

In [22]:
jikan.anime(58567)

{'data': {'mal_id': 58567,
  'url': 'https://myanimelist.net/anime/58567/Ore_dake_Level_Up_na_Ken_Season_2__Arise_from_the_Shadow',
  'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/anime/1448/147351.jpg',
    'small_image_url': 'https://cdn.myanimelist.net/images/anime/1448/147351t.jpg',
    'large_image_url': 'https://cdn.myanimelist.net/images/anime/1448/147351l.jpg'},
   'webp': {'image_url': 'https://cdn.myanimelist.net/images/anime/1448/147351.webp',
    'small_image_url': 'https://cdn.myanimelist.net/images/anime/1448/147351t.webp',
    'large_image_url': 'https://cdn.myanimelist.net/images/anime/1448/147351l.webp'}},
  'trailer': {'youtube_id': 'DYTZbJ2Y4Yk',
   'url': 'https://www.youtube.com/watch?v=DYTZbJ2Y4Yk',
   'embed_url': 'https://www.youtube.com/embed/DYTZbJ2Y4Yk?enablejsapi=1&wmode=opaque&autoplay=1',
   'images': {'image_url': 'https://img.youtube.com/vi/DYTZbJ2Y4Yk/default.jpg',
    'small_image_url': 'https://img.youtube.com/vi/DYTZbJ2Y4Yk/sdde

In [42]:
x = jikan.top(type='anime', page=1)

In [47]:
x['data'][0]['mal_id']

52991

In [ ]:
def get_top_data():
    pass
    data = jikan.top(type='anime', page=1)
    item = {
        'mal_id': data['data'][0]['mal_id'],
        'title': data['data'][0]['title'],
        'rank': data['data'][0]['rank'],
        'score': data['data'][0]['score'],
        'members': data['data'][0]['members'],
        'url': data['data'][0]['url'],
        'image_url': data['data'][0]['image_url']
    }

In [ ]:
import boto3
import requests

def lambda_handler(event, context):
    # Fetch data from Jikan API
    response = requests.get('https://api.jikan.moe/v4/top/anime')
    data = response.json()

    # Update DynamoDB
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('anime_ratings')

    for anime in data['data']:
        table.put_item(
            Item={
                'anime_id': anime['mal_id'],
                'title': anime['title'],
                'timestamp': anime['published']['from'],
                'score': anime['score']
            }
        )
    return {'status': 'success'}
